## Z3

In [1]:
import itertools
import numpy as np
from z3 import *

e=0.001
# perfectly correlated when x=y, random when x!=y
P_AB_giv_XY = {(0, 0, 0, 0): 1/2,
    (0, 0, 0, 1): 1/4,
    (0, 0, 1, 0): 1/4,
    (0, 0, 1, 1): 1/2,
    (0, 1, 0, 0): 0,
    (0, 1, 0, 1): 1/4,
    (0, 1, 1, 0): 1/4,
    (0, 1, 1, 1): 0,
    (1, 0, 0, 0): 0,
    (1, 0, 0, 1): 1/4,
    (1, 0, 1, 0): 1/4,
    (1, 0, 1, 1): 0,
    (1, 1, 0, 0): 1/2,
    (1, 1, 0, 1): 1/4,
    (1, 1, 1, 0): 1/4,
    (1, 1, 1, 1): 1/2}

sum(P_AB_giv_XY.values()) == 4.0

True

In [2]:
card_l = 2
card_A = 2
card_X = 2
card_B = 2
card_Y = 2

P_l = [Real(f'P_l({l})') for l in range(card_l)]
P_A_giv_X_l = [[[Real(f'P_A_giv_X_l({a},{x},{l})') for l in range(card_l)] for x in range(card_X)] for a in range(card_A)]
P_B_giv_Y_l = [[[Real(f'P_B_giv_Y_l({b},{y},{l})') for l in range(card_l)] for y in range(card_Y)] for b in range(card_B)]

s = Solver()


constraints:

In [3]:
# every probabilitiy value is in [0,1]
for l in range(card_l):
    s.add(  P_l[l] >= 0+e, 
            P_l[l] <= 1-e)
# P_l sum to 1
s.add(  Sum(P_l) <= 1+e, 
        Sum(P_l) >= 1-e)

# every probabilitiy value is in [0,1]
for a,x,l in np.ndindex(card_A, card_X, card_l):
    s.add(  P_A_giv_X_l[a][x][l] >= 0-e, 
            P_A_giv_X_l[a][x][l] <= 1+e)

# P_A_giv_X_l sum to 1
for x,l in np.ndindex(card_X, card_l):
    s.add(  Sum([P_A_giv_X_l[a][x][l] for a in range(card_A)]) <= 1+e,
            Sum([P_A_giv_X_l[a][x][l] for a in range(card_A)]) >= 1-e)  

# every probabilitiy value is in [0,1]
for b,y,l in np.ndindex(card_B, card_Y, card_l):
    s.add(  P_B_giv_Y_l[b][y][l] >= 0-e,
            P_B_giv_Y_l[b][y][l] <= 1+e) 
# P_B_giv_Y_l sum to 1
for y,l in np.ndindex(card_Y, card_l):
    s.add(  Sum([P_B_giv_Y_l[b][y][l] for b in range(card_B)]) <= 1+e, 
            Sum([P_B_giv_Y_l[b][y][l] for b in range(card_B)]) >= 1-e)

In [4]:
# Sturctual equation
for a, b, x, y in np.ndindex(card_A, card_B, card_X, card_Y):
    sum_of_products = Sum([P_A_giv_X_l[a][x][l] * P_B_giv_Y_l[b][y][l]*P_l[l] for l in range(card_l)])
    s.add(  sum_of_products - RealVal(P_AB_giv_XY[a, b, x, y]) <= 0+e, 
            sum_of_products - RealVal(P_AB_giv_XY[a, b, x, y]) >= 0-e)

In [5]:
print(s)

[P_l(0) >= 1/1000,
 P_l(0) <= 999/1000,
 P_l(1) >= 1/1000,
 P_l(1) <= 999/1000,
 P_l(0) + P_l(1) <= 1001/1000,
 P_l(0) + P_l(1) >= 999/1000,
 P_A_giv_X_l(0,0,0) >= -1/1000,
 P_A_giv_X_l(0,0,0) <= 1001/1000,
 P_A_giv_X_l(0,0,1) >= -1/1000,
 P_A_giv_X_l(0,0,1) <= 1001/1000,
 P_A_giv_X_l(0,1,0) >= -1/1000,
 P_A_giv_X_l(0,1,0) <= 1001/1000,
 P_A_giv_X_l(0,1,1) >= -1/1000,
 P_A_giv_X_l(0,1,1) <= 1001/1000,
 P_A_giv_X_l(1,0,0) >= -1/1000,
 P_A_giv_X_l(1,0,0) <= 1001/1000,
 P_A_giv_X_l(1,0,1) >= -1/1000,
 P_A_giv_X_l(1,0,1) <= 1001/1000,
 P_A_giv_X_l(1,1,0) >= -1/1000,
 P_A_giv_X_l(1,1,0) <= 1001/1000,
 P_A_giv_X_l(1,1,1) >= -1/1000,
 P_A_giv_X_l(1,1,1) <= 1001/1000,
 P_A_giv_X_l(0,0,0) + P_A_giv_X_l(1,0,0) <= 1001/1000,
 P_A_giv_X_l(0,0,0) + P_A_giv_X_l(1,0,0) >= 999/1000,
 P_A_giv_X_l(0,0,1) + P_A_giv_X_l(1,0,1) <= 1001/1000,
 P_A_giv_X_l(0,0,1) + P_A_giv_X_l(1,0,1) >= 999/1000,
 P_A_giv_X_l(0,1,0) + P_A_giv_X_l(1,1,0) <= 1001/1000,
 P_A_giv_X_l(0,1,0) + P_A_giv_X_l(1,1,0) >= 999/1000,
 P_A

In [6]:
s.check()

In [ ]:
if s.check() == sat:
    print("Solution exists:")
    m = s.model()
    for l in range(card_l):
        print(f"P_l[{l}]:", m.evaluate(P_l[l]).as_decimal(5))
    for a, x, l in np.ndindex(card_A, card_X, card_l):
        print(f"P_A_giv_X_l[{a},{x},{l}]:", m.evaluate(P_A_giv_X_l[a][x][l]).as_decimal(5))
    for b, y, l in np.ndindex(card_B, card_Y, card_l):
        print(f"P_B_giv_Y_l[{b},{y},{l}]:", m.evaluate(P_B_giv_Y_l[b][y][l]).as_decimal(5))
else:
    print("No solution exists.")